In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
from astropy.io import fits
import pandas as pd
from astropy.stats import sigma_clip

if os.path.basename(os.getcwd()) != "RADIANT":
    print("Changing directory to RADIANT")
    os.chdir("..")
from MachineLearning.process import crop_normalize
print(os.getcwd())

Changing directory to RADIANT


2024-05-24 12:52:16.350876: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-24 12:52:16.381917: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-24 12:52:16.641843: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-24 12:52:17.658963: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-24 12:52:22.253728: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

/home/dylan/RADIANT


In [6]:
# fits_folder = "/mnt/c/Users/dylan/Documents/downloaded_images/same_pixels/"
fits_folder = "downloaded_images/same_pixels/"
radcat = pd.read_csv("ConstructData/RADCAT_ML.csv", index_col=0)
channels = {
    "FIRST":0,
    "LOFAR":1,
    "NVSS":2,
}

def show_images(index, sigma=3, norm=True, apply_clip=True, maxiters=5):
# def show_images(index, sigma_lower=3, sigma_upper=3, norm=True, apply_clip=True):
    fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    for file in [x for x in os.listdir(fits_folder) if x.startswith(f"{index}_")]:
        channel = channels[file.split("_")[1]]
        morph = radcat.loc[index, "Type"]
        
        image = fits.getdata(fits_folder + file)
        X = crop_normalize(X=np.expand_dims(image, axis=-1), crop_size=128, names=[index], only_crop=not norm)
        image = X[0,:,:,0]
        
        if apply_clip:        
            mask = sigma_clip(image, maxiters=maxiters, sigma=sigma).mask
            # mask = sigma_clip(image, maxiters=maxiters, sigma_lower=sigma_lower, sigma_upper=sigma_upper).mask
            image[~ mask] = None
        
        im = ax[channel].imshow(image, cmap="viridis")
        fig.colorbar(im, ax=ax[channel])
        ax[channel].set_title(file.split("_")[1])
        fig.suptitle(f"{index} - {morph}")
        
    
interact(show_images, index=radcat.index, sigma=(0.001, 3, 0.001), maxiters=5)
# interact(show_images, index=radcat.index, sigma_lower=(0, 100, 1), sigma_upper=(0, 100, 1))
#! 1660 is an issue
#! 982 is hidden
#! 1723 FIRST/LOFAR hard for Sigma Clipping

interactive(children=(Dropdown(description='index', options=(803, 817, 845, 846, 850, 874, 886, 904, 936, 944,…

<function __main__.show_images(index, sigma=3, norm=True, apply_clip=True, maxiters=5)>

In [7]:
# fits_folder = "/mnt/c/Users/dylan/Documents/downloaded_images/same_pixels/"
fits_folder = "downloaded_images/same_pixels/"
radcat = pd.read_csv("ConstructData/RADCAT_ML.csv", index_col=0)
channels = {
    "FIRST":0,
    "LOFAR":1,
    "NVSS":2,
}

def show_images(index, sigma=3, norm=True, apply_clip=True, maxiters=5):
# def show_images(index, sigma_lower=3, sigma_upper=3, norm=True, apply_clip=True):
    fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    files = []
    channel = []
    for file in [x for x in os.listdir(fits_folder) if x.startswith(f"{index}_")]:
        channel.append(channels[file.split("_")[1]])
        morph = radcat.loc[index, "Type"]
        
        image = fits.getdata(fits_folder + file)
        
        files.append(image)

    files = np.array(files)
    
    files = np.expand_dims(files, axis=-1)

    files = crop_normalize(X=files, crop_size=128, names=[index], only_crop=not norm)
    
    if apply_clip:        
        mask = sigma_clip(files, maxiters=maxiters, sigma=sigma, axis= (1,2) ).mask
        files[~ mask] = None

    for i, c in enumerate(channel):
        image = files[i]
        
        im = ax[c].imshow(image, cmap="viridis")
        fig.colorbar(im, ax=ax[c])
        ax[c].set_title(list(channels.keys())[c])
        fig.suptitle(f"{index} - {morph}")

interact(show_images, index=radcat.index, sigma=(0.001, 3, 0.001), maxiters=5)
# interact(show_images, index=radcat.index, sigma_lower=(0, 100, 1), sigma_upper=(0, 100, 1))
#! 1660 is an issue
#! 982 is hidden

interactive(children=(Dropdown(description='index', options=(803, 817, 845, 846, 850, 874, 886, 904, 936, 944,…

<function __main__.show_images(index, sigma=3, norm=True, apply_clip=True, maxiters=5)>